In [3]:
# Huấn luyện AI (Model Training)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

# Bước 1: Tải dữ liệu
url = "https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv"
data = pd.read_csv(url)
print("Dữ liệu ban đầu:")
print(data.head())

# Bước 2: Xử lý dữ liệu
# Xóa hàng có giá trị NaN
data.dropna(inplace=True)

# Chuyển đổi cột 'ocean_proximity' sang dạng số
data = pd.get_dummies(data, columns=['ocean_proximity'], drop_first=True)

# Chia dữ liệu thành đầu vào (X) và đầu ra (y)
X = data.drop(columns=['median_house_value'])
y = data['median_house_value']

# Chia thành tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bước 3: Huấn luyện mô hình
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Bước 4: Đánh giá mô hình
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    #Sai số trung bình tuyệt đối
    mae = mean_absolute_error(y_test, y_pred)

    # Sai số bình phương trung bình
    mse = mean_squared_error(y_test, y_pred)

    # Sai số phần trăm trung bình tuyệt đối
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

   # Độ chính xác theo R2
    r2 = r2_score(y_test, y_pred)

    print(f"{model.__class__.__name__} - MAE: {mae}, MSE: {mse}")
    print(f"  - R² Score: {r2:.4f}")
    print(f"  - MAPE: {mape:.2f}%")  # Tính sai số phần trăm

evaluate_model(lr_model, X_test, y_test)
evaluate_model(rf_model, X_test, y_test)

# Bước 5: Dự đoán giá nhà
sample_house = np.array(X_test.iloc[0]).reshape(1, -1)
predicted_price = rf_model.predict(sample_house)
print(f"Giá nhà dự đoán: {predicted_price[0]}")

import joblib

# Lưu mô hình RandomForest đã huấn luyện
joblib.dump(rf_model, "house_price_model.pkl")
print("Mô hình đã được lưu!")


Dữ liệu ban đầu:
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY  
LinearRegression - MAE: 50413.4333081006, MSE: 4802173538.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Mô hình đã được lưu!


In [ ]:
# Viết APP Streamlit

%%writefile app.py
import streamlit as st
import pandas as pd
import joblib

# Tải mô hình
model = joblib.load("house_price_model.pkl")

# Tiêu đề ứng dụng
st.title("🏡 Dự đoán giá nhà với Machine Learning")

# Tạo form nhập dữ liệu
st.sidebar.header("Nhập thông tin căn nhà")

def user_input_features():
    longitude = st.sidebar.number_input("Longitude", value=-118.25)
    latitude = st.sidebar.number_input("Latitude", value=34.05)
    housing_median_age = st.sidebar.number_input("Housing Median Age", value=30)
    total_rooms = st.sidebar.number_input("Total Rooms", value=5000)
    total_bedrooms = st.sidebar.number_input("Total Bedrooms", value=1000)
    population = st.sidebar.number_input("Population", value=1500)
    households = st.sidebar.number_input("Households", value=500)
    median_income = st.sidebar.number_input("Median Income", value=3.5)

    # One-hot encoding cho 'ocean_proximity'
    ocean_proximity_inland = st.sidebar.checkbox("Inland", value=False)
    ocean_proximity_island = st.sidebar.checkbox("Island", value=False)
    ocean_proximity_near_bay = st.sidebar.checkbox("Near Bay", value=False)
    ocean_proximity_near_ocean = st.sidebar.checkbox("Near Ocean", value=False)

    data = {
        "longitude": longitude,
        "latitude": latitude,
        "housing_median_age": housing_median_age,
        "total_rooms": total_rooms,
        "total_bedrooms": total_bedrooms,
        "population": population,
        "households": households,
        "median_income": median_income,
        "ocean_proximity_INLAND": int(ocean_proximity_inland),
        "ocean_proximity_ISLAND": int(ocean_proximity_island),
        "ocean_proximity_NEAR BAY": int(ocean_proximity_near_bay),
        "ocean_proximity_NEAR OCEAN": int(ocean_proximity_near_ocean),
    }

    return pd.DataFrame(data, index=[0])

# Lấy dữ liệu từ form
input_df = user_input_features()

# Dự đoán giá nhà
if st.button("Dự đoán giá nhà 💰"):
    prediction = model.predict(input_df)
    st.subheader(f"📢 Giá nhà dự đoán: ${prediction[0]:,.2f}")


Overwriting app.py


In [1]:
# Cần đăng nhập trên ngrok để lấy autoken! (Need to login on ngrok to get autoken!)
# Đây là autoken của tôi, bạn cần tự lấy autoken thay vào! (This is my autoken, you have to auto get autoken!)

!ngrok authtoken "2u7Ish5dUhqNabY4jlN7DkJyd01_3ZusTZZGnFgCKNx1UYgV"

/bin/bash: line 1: ngrok: command not found


In [ ]:
# Chạy app (Run App)
from pyngrok import ngrok
public_url = ngrok.connect(8501, "http")  # Xóa tham số "port="
print(f"Truy cập Streamlit tại: {public_url}")

!streamlit run app.py


Truy cập Streamlit tại: NgrokTunnel: "https://f3d5-34-73-170-62.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.73.170.62:8501



  Stopping...
  Stopping...
Traceback (most recent call last):
  File "/usr/local/bin/streamlit", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1161, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1092, in main
    ctx.exit()
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 697, in exit
    def exit(self, code: int = 0) -> "te.NoReturn":

  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 44, in signal_handler
    server.stop()
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/server/server.py", line 470, in stop
    self._runtime.stop()
  File "/usr/local/lib/python3.11/dist-packages/streamlit/runtime/runtime.py", line 337, in stop
    async_objs.eventloop.call_soon_threadsafe(stop_on_eventloop)
  File "/usr/lib/python3.11/asyncio/b